In [2]:
import torch

torch.cuda.empty_cache()

In [1]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11s-seg.pt")  # load a pretrained model (recommended for training)

In [3]:
DATASET_DIR = "../datasets/DOOM_yolo_seg"


In [ ]:
model.train(
    data=f"{DATASET_DIR}/vizdoom.yaml",
    epochs=150,
    batch=16,
    device=[0],
    patience=10,
    workers=6
)

New https://pypi.org/project/ultralytics/8.3.167 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.162 🚀 Python-3.12.9 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/DOOM_yolo_seg/vizdoom.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train12, nbs=64

train: Scanning /home/kiraprint/Projects/DOOM-RL-speedrun/segmentation/datasets/DOOM_yolo_seg/rgb.cache... 4840 images, 4840 backgrounds, 0 corrupt: 100%|██████████| 9680/9680 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 79.6±11.5 MB/s, size: 98.7 KB)


val: Scanning /home/kiraprint/Projects/DOOM-RL-speedrun/segmentation/datasets/DOOM_yolo_seg/rgb.cache... 4840 images, 4840 backgrounds, 0 corrupt: 100%|██████████| 9680/9680 [00:00<?, ?it/s]


Plotting labels to /home/kiraprint/Projects/DOOM-RL-speedrun/runs/segment/train12/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 6 dataloader workers
Logging results to /home/kiraprint/Projects/DOOM-RL-speedrun/runs/segment/train12
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      1/150      5.14G      1.069      2.349      2.388        1.2        248        640: 100%|██████████| 605/605 [04:22<00:00,  2.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  64%|██████▎   | 193/303 [03:22<1:08:01, 37.10s/it]

In [ ]:
model.save(f"{DATASET_DIR}/trained_model.pt")

In [ ]:
import torch
import torch.nn as nn


class LSTM_Predictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM_Predictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size)
        )

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.mlp(out[:, -1, :])  # Get the last time step's output
        return out


class LSTM_Predictor_Trainer:
    def __init__(self, model, input_size, hidden_size=128, num_layers=2):
        self.model = model
        self.lstm_predictor = LSTM_Predictor(input_size, hidden_size, num_layers)

    def train(self, train_loader, epochs=10, lr=0.001):
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(self.lstm_predictor.parameters(), lr=lr)

        for epoch in range(epochs):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = self.lstm_predictor(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")